###### General Steps to Follow
1. Importing Packages
1. Criteria 1 - Poses
1. Criteria 2 - Poses
1. Criteria 3 - Poses
1. Criteria 4 - Poses
1. Full Test

# -------------------------------------------------------------------------------------------------------------

## 1) Importing Packages

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf

2024-04-28 23:04:14.079185: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 23:04:14.925535: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# -------------------------------------------------------------------------------------------------------------

In [2]:
def evaluate(output, y_true):
    y_pred = np.zeros((output.shape[0],1), dtype = "int")
    y_pred[(output > 0.5)] = 1
    y_pred = y_pred.reshape(y_pred.shape[0])
    print("accuracy =", accuracy_score(y_true, y_pred))

In [3]:
def reshape_input_sequence(data, shape):
    new_data = []
    for training_example in data:
        new_data.append(training_example[0:shape])
    new_data = np.array(new_data)
    return new_data

## 2) Criteria 1 - Poses

#### Loading training data

In [6]:
criteria1_data_poses = np.load("../data/processed/lateral_raise/poses/training_data_lateral_raise_poses_criteria_1.npy", allow_pickle = True)

In [7]:
len(criteria1_data_poses)

573

#### Getting X_train, X_test, y_train, y_test

In [8]:
X = np.vstack(criteria1_data_poses[:, 0])
y = np.vstack(criteria1_data_poses[:, 1])
X_train_criteria1_poses, X_test_criteria1_poses, y_train_criteria1_poses, y_test_criteria1_poses = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [9]:
print("X_train_criteria1_poses shape:", X_train_criteria1_poses.shape)
print("y_train_criteria1_poses shape:", y_train_criteria1_poses.shape)
print("X_test_criteria1_poses shape:", X_test_criteria1_poses.shape)
print("y_test_criteria1_poses shape:", y_test_criteria1_poses.shape)

X_train_criteria1_poses shape: (458, 600, 24)
y_train_criteria1_poses shape: (458, 1)
X_test_criteria1_poses shape: (115, 600, 24)
y_test_criteria1_poses shape: (115, 1)


In [56]:
num = random.randint(0,457)
X_train_criteria1_poses[num][90]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

#### Reducing the length of the input sequence in X_train, X_test

In [58]:
new_length = 150
X_train_criteria1_poses = reshape_input_sequence(X_train_criteria1_poses, new_length)
X_test_criteria1_poses = reshape_input_sequence(X_test_criteria1_poses, new_length)
print("X_train_criteria1_poses shape:", X_train_criteria1_poses.shape)
print("y_train_criteria1_poses shape:", y_train_criteria1_poses.shape)
print("X_test_criteria1_poses shape:", X_test_criteria1_poses.shape)
print("y_test_criteria1_poses shape:", y_test_criteria1_poses.shape)

X_train_criteria1_poses shape: (458, 150, 24)
y_train_criteria1_poses shape: (458, 1)
X_test_criteria1_poses shape: (115, 150, 24)
y_test_criteria1_poses shape: (115, 1)


#### Building and training the model

In [59]:
criteria1_poses_model = Sequential(
    [
        GRU(units=256, input_shape=(X_train_criteria1_poses.shape[1], X_train_criteria1_poses.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 64, return_sequences=True),
        BatchNormalization(),
        GRU(units = 32),
        BatchNormalization(),
        #Dense(256, activation = "relu"),
        #BatchNormalization(),
        #Dense(128, activation = "relu"),
        #BatchNormalization(),
        #Dense(64, activation = "relu"),
         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], 
    name="criteria1_poses_model"
)

criteria1_poses_model.summary()

2024-04-28 23:12:04.503974: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 23:12:04.578016: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model inste

Model: "criteria1_poses_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 150, 256)       │       216,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 150, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 150, 128)       │       148,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 150, 64)        │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 150, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 413,409 (1.58 MB)

 Trainable params: 412,449 (1.57 MB)

 Non-trainable params: 960 (3.75 KB)

In [90]:
criteria1_poses_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0001),
                              loss = BinaryCrossentropy(from_logits = True))

In [94]:
criteria1_poses_model.fit(X_train_criteria1_poses, y_train_criteria1_poses, epochs = 5, batch_size = 32)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 283ms/step - accuracy: 0.9783 - loss: 0.0419
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 280ms/step - accuracy: 0.9621 - loss: 0.0686
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 285ms/step - accuracy: 0.9697 - loss: 0.0550
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 269ms/step - accuracy: 0.9910 - loss: 0.0293
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 283ms/step - accuracy: 0.9965 - loss: 0.0272


#### Saving the model

In [98]:
criteria1_poses_model.save("../models/other/lateral_raise/poses/lateral_raise_criteria1_poses_model_v2.h5")

#### Loading the model

In [89]:
criteria1_poses_model = tf.keras.models.load_model("../models/other/lateral_raise/poses/lateral_raise_criteria1_poses_model_v1.h5")

#### Evaluating the model

###### Train Data

In [96]:
y_pred = criteria1_poses_model.predict(X_train_criteria1_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria1_poses)

15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step
accuracy = 0.9235807860262009


###### Test Data

In [95]:
y_pred = criteria1_poses_model.predict(X_test_criteria1_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria1_poses)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
accuracy = 0.9478260869565217


# -------------------------------------------------------------------------------------------------------------

## 3) Criteria 2 - Poses

#### Loading training data

In [99]:
criteria2_data_poses = np.load("../data/processed/lateral_raise/poses/training_data_lateral_raise_poses_criteria_2.npy", allow_pickle = True)

In [100]:
len(criteria2_data_poses)

681

#### Getting X_train, X_test, y_train, y_test

In [101]:
X = np.vstack(criteria2_data_poses[:, 0])
y = np.vstack(criteria2_data_poses[:, 1])
X_train_criteria2_poses, X_test_criteria2_poses, y_train_criteria2_poses, y_test_criteria2_poses = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [102]:
print("X_train_criteria2_poses shape:", X_train_criteria2_poses.shape)
print("y_train_criteria2_poses shape:", y_train_criteria2_poses.shape)
print("X_test_criteria2_poses shape:", X_test_criteria2_poses.shape)
print("y_test_criteria2_poses shape:", y_test_criteria2_poses.shape)

X_train_criteria2_poses shape: (544, 600, 24)
y_train_criteria2_poses shape: (544, 1)
X_test_criteria2_poses shape: (137, 600, 24)
y_test_criteria2_poses shape: (137, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [103]:
new_length = 150
X_train_criteria2_poses = reshape_input_sequence(X_train_criteria2_poses, new_length)
X_test_criteria2_poses = reshape_input_sequence(X_test_criteria2_poses, new_length)
print("X_train_criteria2_poses shape:", X_train_criteria2_poses.shape)
print("y_train_criteria2_poses shape:", y_train_criteria2_poses.shape)
print("X_test_criteria2_poses shape:", X_test_criteria2_poses.shape)
print("y_test_criteria2_poses shape:", y_test_criteria2_poses.shape)

X_train_criteria2_poses shape: (544, 150, 24)
y_train_criteria2_poses shape: (544, 1)
X_test_criteria2_poses shape: (137, 150, 24)
y_test_criteria2_poses shape: (137, 1)


#### Building and training the model

In [117]:
criteria2_poses_model = Sequential(
    [
        GRU(units=128, input_shape=(X_train_criteria2_poses.shape[1], X_train_criteria2_poses.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 32),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], name = "criteria2_poses_model"
)
criteria2_poses_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria2_poses_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_8 (GRU)                     │ (None, 150, 128)       │        59,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ (None, 150, 128)       │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_10 (GRU)                    │ (None, 150, 128)       │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_11 (GRU)                    │ (None, 32)             │        15,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 274,529 (1.05 MB)

 Trainable params: 273,697 (1.04 MB)

 Non-trainable params: 832 (3.25 KB)

In [140]:
criteria2_poses_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.00001),
                              loss = BinaryCrossentropy(from_logits = True))

In [141]:
criteria2_poses_model.fit(X_train_criteria2_poses, y_train_criteria2_poses, epochs = 10, batch_size = 32)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 234ms/step - accuracy: 0.9755 - loss: 0.0700
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 235ms/step - accuracy: 0.9792 - loss: 0.0550
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 226ms/step - accuracy: 0.9821 - loss: 0.0622
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 230ms/step - accuracy: 0.9603 - loss: 0.0822
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 225ms/step - accuracy: 0.9700 - loss: 0.0881
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 243ms/step - accuracy: 0.9575 - loss: 0.0756
Epoch 7/10
16/17 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.9699 - loss: 0.0804

KeyboardInterrupt: 

#### Saving the model

In [146]:
criteria2_poses_model.save("../models/other/lateral_raise/poses/lateral_raise_criteria2_poses_model_v2.h5")

#### Loading the model

In [283]:
criteria2_poses_model = tf.keras.models.load_model("../models/other/lateral_raise/poses/lateral_raise_criteria2_poses_model_v1.h5")

#### Evaluating the model

###### Train Data

In [142]:
y_pred = criteria2_poses_model.predict(X_train_criteria2_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria2_poses)

17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step
accuracy = 0.9503676470588235


###### Test Data

In [143]:
y_pred = criteria2_poses_model.predict(X_test_criteria2_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria2_poses)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 206ms/step
accuracy = 0.927007299270073


In [145]:
#y_pred

# -------------------------------------------------------------------------------------------------------------

## 4) Criteria 3 - Poses

#### Loading training data

In [147]:
criteria3_data_poses = np.load("../data/processed/lateral_raise/poses/training_data_lateral_raise_poses_criteria_3.npy", allow_pickle = True)

In [148]:
len(criteria3_data_poses)

466

#### Getting X_train, X_test, y_train, y_test

In [149]:
X = np.vstack(criteria3_data_poses[:, 0])
y = np.vstack(criteria3_data_poses[:, 1])
X_train_criteria3_poses, X_test_criteria3_poses, y_train_criteria3_poses, y_test_criteria3_poses = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [150]:
print("X_train_criteria3_poses shape:", X_train_criteria3_poses.shape)
print("y_train_criteria3_poses shape:", y_train_criteria3_poses.shape)
print("X_test_criteria3_poses shape:", X_test_criteria3_poses.shape)
print("y_test_criteria3_poses shape:", y_test_criteria3_poses.shape)

X_train_criteria3_poses shape: (372, 600, 24)
y_train_criteria3_poses shape: (372, 1)
X_test_criteria3_poses shape: (94, 600, 24)
y_test_criteria3_poses shape: (94, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [151]:
new_length = 150
X_train_criteria3_poses = reshape_input_sequence(X_train_criteria3_poses, new_length)
X_test_criteria3_poses = reshape_input_sequence(X_test_criteria3_poses, new_length)
print("X_train_criteria3_poses shape:", X_train_criteria3_poses.shape)
print("y_train_criteria3_poses shape:", y_train_criteria3_poses.shape)
print("X_test_criteria3_poses shape:", X_test_criteria3_poses.shape)
print("y_test_criteria3_poses shape:", y_test_criteria3_poses.shape)

X_train_criteria3_poses shape: (372, 150, 24)
y_train_criteria3_poses shape: (372, 1)
X_test_criteria3_poses shape: (94, 150, 24)
y_test_criteria3_poses shape: (94, 1)


#### Building and training the model

In [161]:
criteria3_poses_model = Sequential(
    [
        GRU(units=256, input_shape=(X_train_criteria3_poses.shape[1], X_train_criteria3_poses.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        GRU(units = 190, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 150, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 64, return_sequences=True),
        BatchNormalization(),
        GRU(units = 20),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], name = "criteria3_poses_model"
)
criteria3_poses_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria3_poses_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_16 (GRU)                    │ (None, 150, 256)       │       216,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 150, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_17 (GRU)                    │ (None, 150, 190)       │       255,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 150, 190)       │           760 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_18 (GRU)                    │ (None, 150, 64)        │        49,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 150, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_19 (GRU)                    │ (None, 20)             │         5,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 20)             │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 528,389 (2.02 MB)

 Trainable params: 527,329 (2.01 MB)

 Non-trainable params: 1,060 (4.14 KB)

In [183]:
criteria3_poses_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.000001),
                              loss = BinaryCrossentropy(from_logits = True))

In [191]:
criteria3_poses_model.fit(X_train_criteria3_poses, y_train_criteria3_poses, epochs = 10, batch_size = 32)

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 306ms/step - accuracy: 0.9818 - loss: 0.0615
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 310ms/step - accuracy: 0.9896 - loss: 0.0399
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 314ms/step - accuracy: 0.9882 - loss: 0.0516
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 302ms/step - accuracy: 0.9980 - loss: 0.0499
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 306ms/step - accuracy: 0.9802 - loss: 0.0597
Epoch 6/10
 1/12 ━━━━━━━━━━━━━━━━━━━━ 3s 329ms/step - accuracy: 1.0000 - loss: 0.0215

KeyboardInterrupt: 

#### Saving the model

In [194]:
criteria3_poses_model.save("../models/other/lateral_raise/poses/lateral_raise_criteria3_poses_model_v2.h5")

#### Loading the model

In [ ]:
criteria3_poses_model = load_model("../models/other/lateral_raise/poses/lateral_raise_criteria3_poses_model_v1")

#### Evaluating the model

###### Train Data

In [192]:
y_pred = criteria3_poses_model.predict(X_train_criteria3_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria3_poses)

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step
accuracy = 0.9408602150537635


###### Test Data

In [193]:
y_pred = criteria3_poses_model.predict(X_test_criteria3_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria3_poses)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
accuracy = 0.9361702127659575


# -------------------------------------------------------------------------------------------------------------

## 5) Criteria 4 - Poses

#### Loading training data

In [195]:
criteria4_data_poses = np.load("../data/processed/lateral_raise/poses/training_data_lateral_raise_poses_criteria_4.npy", allow_pickle = True)

In [196]:
len(criteria4_data_poses)

471

#### Getting X_train, X_test, y_train, y_test

In [197]:
X = np.vstack(criteria4_data_poses[:, 0])
y = np.vstack(criteria4_data_poses[:, 1])
X_train_criteria4_poses, X_test_criteria4_poses, y_train_criteria4_poses, y_test_criteria4_poses = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [198]:
print("X_train_criteria4_poses shape:", X_train_criteria4_poses.shape)
print("y_train_criteria4_poses shape:", y_train_criteria4_poses.shape)
print("X_test_criteria4_poses shape:", X_test_criteria4_poses.shape)
print("y_test_criteria4_poses shape:", y_test_criteria4_poses.shape)

X_train_criteria4_poses shape: (376, 600, 24)
y_train_criteria4_poses shape: (376, 1)
X_test_criteria4_poses shape: (95, 600, 24)
y_test_criteria4_poses shape: (95, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [199]:
new_length = 150
X_train_criteria4_poses = reshape_input_sequence(X_train_criteria4_poses, new_length)
X_test_criteria4_poses = reshape_input_sequence(X_test_criteria4_poses, new_length)
print("X_train_criteria4_poses shape:", X_train_criteria4_poses.shape)
print("y_train_criteria4_poses shape:", y_train_criteria4_poses.shape)
print("X_test_criteria4_poses shape:", X_test_criteria4_poses.shape)
print("y_test_criteria4_poses shape:", y_test_criteria4_poses.shape)

X_train_criteria4_poses shape: (376, 150, 24)
y_train_criteria4_poses shape: (376, 1)
X_test_criteria4_poses shape: (95, 150, 24)
y_test_criteria4_poses shape: (95, 1)


#### Building and training the model

In [50]:
criteria4_poses_model = Sequential(
    [
        LSTM(units=50, input_shape=(X_train_criteria4_poses.shape[1], X_train_criteria4_poses.shape[2])),
        Dense(1, activation = "sigmoid")
    ], name = "criteria4_poses_model"
)
criteria4_poses_model.summary()

Model: "criteria4_poses_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 50)                15000     
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 15051 (58.79 KB)
Trainable params: 15051 (58.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
criteria4_poses_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [52]:
criteria4_poses_model.fit(X_train_criteria4_poses,y_train_criteria4_poses, epochs = 10)

Epoch 1/10
21/21 [==============================] - 5s 187ms/step - loss: 0.6757 - accuracy: 0.6262
Epoch 2/10
21/21 [==============================] - 4s 176ms/step - loss: 0.6660 - accuracy: 0.6262
Epoch 3/10
21/21 [==============================] - 4s 187ms/step - loss: 0.6621 - accuracy: 0.6262
Epoch 4/10
21/21 [==============================] - 4s 198ms/step - loss: 0.6622 - accuracy: 0.6262
Epoch 5/10
21/21 [==============================] - 4s 185ms/step - loss: 0.6621 - accuracy: 0.6262
Epoch 6/10
21/21 [==============================] - 4s 188ms/step - loss: 0.6619 - accuracy: 0.6262
Epoch 7/10
21/21 [==============================] - 4s 184ms/step - loss: 0.6655 - accuracy: 0.6262
Epoch 8/10
21/21 [==============================] - 4s 187ms/step - loss: 0.6626 - accuracy: 0.6262
Epoch 9/10
21/21 [==============================] - 4s 167ms/step - loss: 0.6621 - accuracy: 0.6262
Epoch 10/10
21/21 [==============================] - 4s 179ms/step - loss: 0.6617 - accuracy: 0.6262

#### Saving the model

In [ ]:
criteria4_poses_model.save("model.h5")

#### Loading the model

In [ ]:
loaded_model = load_model('my_model.h5')

#### Evaluating the model

###### Train Data

In [53]:
y_pred = criteria4_poses_model.predict(X_train_criteria4_poses)
evaluate(y_pred, y_train_criteria4_poses)

21/21 [==============================] - 1s 57ms/step
accuracy = 0.6261682242990654


###### Test Data

In [54]:
y_pred = criteria4_poses_model.predict(X_test_criteria4_poses)
evaluate(y_pred, y_test_criteria4_poses)

6/6 [==============================] - 0s 40ms/step
accuracy = 0.577639751552795


# -------------------------------------------------------------------------------------------------------------

## 6) Full Test

In [200]:
def test_models(input_sample):
    input_sample = input_sample.reshape((1, input_sample.shape[0], input_sample.shape[1]))
    y_pred1 = criteria1_poses_model.predict(input_sample)
    y_pred1 = tf.nn.sigmoid(y_pred1)
    y_pred2 = criteria2_poses_model.predict(input_sample)
    y_pred2 = tf.nn.sigmoid(y_pred2)
    y_pred3 = criteria3_poses_model.predict(input_sample)
    y_pred3 = tf.nn.sigmoid(y_pred3)
    print("Criteria1:", y_pred1[0][0].numpy())
    print("Criteria2:", y_pred2[0][0].numpy())
    print("Criteria3:", y_pred3[0][0].numpy())

In [201]:
import random

In [236]:
num = random.randint(0,X_test_criteria4_poses.shape[0]-1)
input_sample = X_test_criteria4_poses[num]
print(y_test_criteria4_poses[num])
test_models(input_sample)

[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Criteria1: 0.5231946
Criteria2: 0.20665324
Criteria3: 0.22147901
